In [1]:
import pandas as pd
import numpy as np

df_reg = pd.read_csv('../resources/data/unprocessed/MRegularSeasonCompactResults.csv')

df_reg = df_reg.loc[(df_reg['Season'] >= 2011) & (df_reg['Season'] != 2020), ].reset_index(drop=True)

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,2011,7,1228,79,1414,65,H,0
1,2011,7,1268,105,1370,76,H,0
2,2011,7,1338,83,1348,75,H,0
3,2011,7,1400,83,1298,52,H,0
4,2011,9,1228,84,1405,45,H,0
...,...,...,...,...,...,...,...,...
62964,2023,132,1104,82,1401,63,N,0
62965,2023,132,1272,75,1222,65,A,0
62966,2023,132,1343,74,1463,65,H,0
62967,2023,132,1345,67,1336,65,N,0


In [2]:
df_teams = pd.read_csv('../resources/data/unprocessed/MTeams.csv')

df_teams

,TeamID,TeamName,FirstD1Season,LastD1Season
0,1101,Abilene Chr,2014,2023
1,1102,Air Force,1985,2023
2,1103,Akron,1985,2023
3,1104,Alabama,1985,2023
4,1105,Alabama A&M,2000,2023
...,...,...,...,...
372,1473,Lindenwood,2023,2023
373,1474,Queens NC,2023,2023
374,1475,Southern Indiana,2023,2023
375,1476,Stonehill,2023,2023


In [3]:
df_reg['WTeamName'] = pd.merge(
    df_reg, df_teams, how='left', 
    left_on=['WTeamID'], right_on=['TeamID']
)['TeamName']

df_reg['LTeamName'] = pd.merge(
    df_reg, df_teams, how='left', 
    left_on=['LTeamID'], right_on=['TeamID']
)['TeamName']

df_reg

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WTeamName,LTeamName
0,2011,7,1228,79,1414,65,H,0,Illinois,UC Irvine
1,2011,7,1268,105,1370,76,H,0,Maryland,Seattle
2,2011,7,1338,83,1348,75,H,0,Pittsburgh,Rhode Island
3,2011,7,1400,83,1298,52,H,0,Texas,Navy
4,2011,9,1228,84,1405,45,H,0,Illinois,Toledo
...,...,...,...,...,...,...,...,...,...,...
62964,2023,132,1104,82,1401,63,N,0,Alabama,Texas A&M
62965,2023,132,1272,75,1222,65,A,0,Memphis,Houston
62966,2023,132,1343,74,1463,65,H,0,Princeton,Yale
62967,2023,132,1345,67,1336,65,N,0,Purdue,Penn St


In [4]:
seasons_list = [season for season in range(2011, 2024) if season != 2020]

seasons_list

[2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]

In [5]:
sr_teams = []

for season in seasons_list:        
    print(season)
    
    df_sr_teams_season = pd.read_html(
        f'https://www.sports-reference.com/cbb/seasons/men/{season}-school-stats.html'
    )[0]

    df_sr_teams_season.columns = [i[1] for i in df_sr_teams_season.columns]

    df_sr_teams_season = df_sr_teams_season.iloc[:, :5]

    df_sr_teams_season = df_sr_teams_season.loc[~df_sr_teams_season['School'].isna(), :].copy()
    df_sr_teams_season = df_sr_teams_season.loc[df_sr_teams_season['School'] != 'School', :].copy()

    df_sr_teams_season['School'] = df_sr_teams_season['School'].str.replace('\xa0NCAA', '')

    df_sr_teams_season = df_sr_teams_season[['School', 'W', 'L']]

    df_sr_teams_season[['W', 'L']] = df_sr_teams_season[['W', 'L']].astype('int32')
    
    df_sr_teams_season.insert(0, 'season', season)

    sr_teams.append(df_sr_teams_season)

df_sr_teams = pd.concat(sr_teams, ignore_index=True)

df_sr_teams

2011
2012
2013
2014
2015
2016
2017
2018
2019
2021
2022
2023


,season,School,W,L
0,2011,Air Force,16,16
1,2011,Akron,23,13
2,2011,Alabama,25,12
3,2011,Alabama A&M,13,15
4,2011,Alabama State,17,18
...,...,...,...,...
4215,2023,Wright State,18,15
4216,2023,Wyoming,9,22
4217,2023,Xavier,27,10
4218,2023,Yale,21,9


In [6]:
df_sr_teams['School'] = df_sr_teams['School'].str.replace('(', '', regex=False).str.replace(
        ')', '', regex=False).str.replace(
        "'", '', regex=False).str.replace(
        '.', '', regex=False)

df_sr_teams

,season,School,W,L
0,2011,Air Force,16,16
1,2011,Akron,23,13
2,2011,Alabama,25,12
3,2011,Alabama A&M,13,15
4,2011,Alabama State,17,18
...,...,...,...,...
4215,2023,Wright State,18,15
4216,2023,Wyoming,9,22
4217,2023,Xavier,27,10
4218,2023,Yale,21,9


In [7]:
# reshape so each team is one row containing all seasons
df_sr_teams = df_sr_teams.pivot(
    index=['School'], 
    columns=['season'], 
    values=['W', 'L']
).reset_index()

df_sr_teams.columns = [f'{i[1]}_{i[0]}'if i[1] else i[0] 
                       for i in df_sr_teams.columns]

df_sr_teams

,School,2011_W,2012_W,2013_W,2014_W,2015_W,2016_W,2017_W,2018_W,2019_W,...,2013_L,2014_L,2015_L,2016_L,2017_L,2018_L,2019_L,2021_L,2022_L,2023_L
0,Abilene Christian,NaN,NaN,NaN,11.0,10.0,13.0,13.0,16.0,27.0,...,NaN,20.0,21.0,18.0,16.0,16.0,7.0,5.0,11.0,17.0
1,Air Force,16.0,13.0,18.0,12.0,14.0,14.0,12.0,12.0,14.0,...,14.0,18.0,17.0,18.0,21.0,19.0,18.0,20.0,18.0,18.0
2,Akron,23.0,22.0,26.0,21.0,21.0,26.0,27.0,14.0,17.0,...,7.0,13.0,14.0,9.0,9.0,18.0,16.0,8.0,10.0,11.0
3,Alabama,25.0,21.0,23.0,13.0,19.0,18.0,19.0,20.0,18.0,...,13.0,19.0,15.0,15.0,15.0,16.0,16.0,7.0,14.0,6.0
4,Alabama A&M,13.0,7.0,11.0,14.0,9.0,11.0,2.0,3.0,5.0,...,20.0,16.0,20.0,18.0,27.0,28.0,27.0,9.0,18.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,Wright State,19.0,13.0,23.0,21.0,11.0,22.0,20.0,25.0,21.0,...,13.0,15.0,20.0,13.0,12.0,10.0,14.0,6.0,14.0,15.0
361,Wyoming,10.0,21.0,20.0,18.0,25.0,14.0,23.0,20.0,8.0,...,14.0,15.0,10.0,18.0,15.0,13.0,24.0,11.0,9.0,22.0
362,Xavier,24.0,23.0,17.0,21.0,23.0,28.0,24.0,29.0,19.0,...,14.0,13.0,14.0,6.0,14.0,6.0,16.0,8.0,13.0,10.0
363,Yale,15.0,19.0,14.0,19.0,22.0,23.0,18.0,16.0,22.0,...,17.0,14.0,10.0,7.0,11.0,15.0,8.0,0.0,12.0,9.0


In [8]:
def sr_seasons_played(row, seasons_list=seasons_list):
    seasons = []
    for season in seasons_list:
        if not np.isnan(row[f'{season}_W'].iloc[0]):
            seasons.append(season)
    
    return seasons

sr_seasons_played(df_sr_teams.iloc[0:1, :])

[2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]

In [9]:
seasons_played = [sr_seasons_played(df_sr_teams.iloc[i:(i+1), :]) for i in range(df_sr_teams.shape[0])]

seasons_played[:10]

[[2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]]

In [10]:
sr_teams_seasons_mapping = dict(zip(df_sr_teams['School'], seasons_played))

sr_teams_seasons_mapping

{'Abilene Christian': [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 'Air Force': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Akron': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alabama': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alabama A&M': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alabama State': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Albany NY': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alcorn State': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'American': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Appalachian State': [2011,
  2012,
  2

In [11]:
df_reg_reshape = pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'team_id': df_reg['WTeamID'], 
                               'team_name': df_reg['WTeamName'], 
                               'opponent_id': df_reg['LTeamID'], 
                               'opponent_name': df_reg['LTeamName'], 
                               'result': np.repeat('win', df_reg.shape[0])})

temp =           pd.DataFrame({'season': df_reg['Season'], 
                               'day_number': df_reg['DayNum'], 
                               'team_id': df_reg['LTeamID'], 
                               'team_name': df_reg['LTeamName'], 
                               'opponent_id': df_reg['WTeamID'], 
                               'opponent_name': df_reg['WTeamName'], 
                               'result': np.repeat('loss', df_reg.shape[0])})

df_reg_reshape = pd.concat([df_reg_reshape, temp], axis=0).reset_index(drop=True)

df_reg_reshape

,season,day_number,team_id,team_name,opponent_id,opponent_name,result
0,2011,7,1228,Illinois,1414,UC Irvine,win
1,2011,7,1268,Maryland,1370,Seattle,win
2,2011,7,1338,Pittsburgh,1348,Rhode Island,win
3,2011,7,1400,Texas,1298,Navy,win
4,2011,9,1228,Illinois,1405,Toledo,win
...,...,...,...,...,...,...,...
125933,2023,132,1401,Texas A&M,1104,Alabama,loss
125934,2023,132,1222,Houston,1272,Memphis,loss
125935,2023,132,1463,Yale,1343,Princeton,loss
125936,2023,132,1336,Penn St,1345,Purdue,loss


In [12]:
# get each teams record
df_records = df_reg_reshape.groupby(
    ['season', 'team_id', 'team_name', 'result']
)['result'].aggregate(
    ['count']
).reset_index().pivot(
    index=['season', 'team_id', 'team_name'], 
    columns=['result'], values=['count']
).reset_index()

df_records.columns = [i[1] if i[1] else i[0] for i in df_records.columns]

# fix winless and undefeated teams
df_records.loc[df_records['loss'].isna(), 'loss'] = 0
df_records.loc[df_records['win'].isna(), 'win'] = 0

df_records[['loss', 'win']] = df_records[['loss', 'win']].astype('int32')

# switch order of wins and losses
df_records.insert(df_records.columns.get_loc('loss'), 'win', df_records.pop('win'))


# reshape so each team is one row containing all seasons
df_records = df_records.pivot(
    index=['team_id', 'team_name'], 
    columns=['season'], 
    values=['win', 'loss']
).reset_index()

df_records.columns = [f'{i[1]}_{i[0]}s' if i[0] == 'win' else f'{i[1]}_{i[0]}es' 
                      if i[1] else i[0] for i in df_records.columns]

df_records

,team_id,team_name,2011_wins,2012_wins,2013_wins,2014_wins,2015_wins,2016_wins,2017_wins,2018_wins,...,2013_losses,2014_losses,2015_losses,2016_losses,2017_losses,2018_losses,2019_losses,2021_losses,2022_losses,2023_losses
0,1101,Abilene Chr,NaN,NaN,NaN,2.0,7.0,9.0,9.0,12.0,...,NaN,19.0,21.0,18.0,16.0,15.0,6.0,4.0,10.0,17.0
1,1102,Air Force,14.0,11.0,15.0,10.0,12.0,12.0,10.0,10.0,...,13.0,18.0,17.0,18.0,21.0,19.0,18.0,20.0,18.0,18.0
2,1103,Akron,22.0,21.0,25.0,21.0,20.0,25.0,25.0,13.0,...,6.0,12.0,14.0,8.0,8.0,18.0,16.0,8.0,9.0,11.0
3,1104,Alabama,21.0,21.0,20.0,12.0,17.0,18.0,19.0,19.0,...,12.0,19.0,14.0,14.0,14.0,15.0,15.0,6.0,13.0,5.0
4,1105,Alabama A&M,10.0,5.0,9.0,12.0,8.0,10.0,2.0,3.0,...,20.0,16.0,20.0,18.0,27.0,28.0,27.0,9.0,18.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360,1473,Lindenwood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.0
361,1474,Queens NC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.0
362,1475,Southern Indiana,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0
363,1476,Stonehill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0


In [13]:
def kaggle_seasons_played(row, seasons_list=seasons_list):
    seasons = []
    for season in seasons_list:
        if not np.isnan(row[f'{season}_wins'].iloc[0]):
            seasons.append(season)
    
    return seasons

kaggle_seasons_played(df_records.iloc[0:1, :])

[2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]

In [14]:
seasons_played = [kaggle_seasons_played(df_records.iloc[i:(i+1), :]) for i in range(df_records.shape[0])]

seasons_played[:10]

[[2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 [2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023]]

In [15]:
kaggle_teams_seasons_mapping = dict(zip(df_records['team_name'], seasons_played))

kaggle_teams_seasons_mapping

{'Abilene Chr': [2014, 2015, 2016, 2017, 2018, 2019, 2021, 2022, 2023],
 'Air Force': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Akron': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alabama': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alabama A&M': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alabama St': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'SUNY Albany': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Alcorn St': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'American Univ': [2011,
  2012,
  2013,
  2014,
  2015,
  2016,
  2017,
  2018,
  2019,
  2021,
  2022,
  2023],
 'Appalachian St': [2011,
  2012,
  2013,
  2

In [16]:
team_guess = df_records[['team_id', 'team_name']].merge(
    df_sr_teams['School'], how='inner', 
    left_on=['team_name'], 
    right_on=['School']).reset_index(drop=True).rename(
    columns={'team_name': 'kaggle_team', 'School': 'sr_team'}
)

team_guess['score'] = 100

team_guess

,team_id,kaggle_team,sr_team,score
0,1102,Air Force,Air Force,100
1,1103,Akron,Akron,100
2,1104,Alabama,Alabama,100
3,1105,Alabama A&M,Alabama A&M,100
4,1112,Arizona,Arizona,100
...,...,...,...,...
213,1471,UC San Diego,UC San Diego,100
214,1473,Lindenwood,Lindenwood,100
215,1474,Queens NC,Queens NC,100
216,1475,Southern Indiana,Southern Indiana,100


In [17]:
df_sr_unmatched = df_sr_teams.loc[~df_sr_teams['School'].isin(team_guess['sr_team']), 
                                  :].reset_index(drop=True)

df_sr_unmatched

,School,2011_W,2012_W,2013_W,2014_W,2015_W,2016_W,2017_W,2018_W,2019_W,...,2013_L,2014_L,2015_L,2016_L,2017_L,2018_L,2019_L,2021_L,2022_L,2023_L
0,Abilene Christian,NaN,NaN,NaN,11.0,10.0,13.0,13.0,16.0,27.0,...,NaN,20.0,21.0,18.0,16.0,16.0,7.0,5.0,11.0,17.0
1,Alabama State,17.0,12.0,10.0,19.0,19.0,14.0,8.0,8.0,12.0,...,22.0,13.0,10.0,17.0,23.0,23.0,19.0,14.0,21.0,23.0
2,Albany NY,16.0,19.0,24.0,19.0,24.0,24.0,21.0,22.0,12.0,...,11.0,15.0,9.0,9.0,14.0,10.0,20.0,9.0,18.0,23.0
3,Alcorn State,4.0,10.0,10.0,12.0,6.0,15.0,18.0,11.0,10.0,...,24.0,19.0,26.0,15.0,14.0,21.0,21.0,13.0,17.0,14.0
4,American,22.0,20.0,10.0,20.0,17.0,12.0,8.0,6.0,15.0,...,20.0,13.0,16.0,19.0,22.0,24.0,15.0,6.0,22.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Western Kentucky,16.0,16.0,20.0,20.0,20.0,18.0,15.0,27.0,20.0,...,16.0,12.0,12.0,16.0,17.0,11.0,14.0,8.0,13.0,16.0
143,Western Michigan,21.0,14.0,22.0,23.0,20.0,13.0,16.0,17.0,8.0,...,13.0,10.0,14.0,19.0,16.0,15.0,24.0,16.0,23.0,23.0
144,Wichita State,29.0,27.0,30.0,35.0,30.0,26.0,31.0,25.0,22.0,...,9.0,1.0,5.0,9.0,5.0,8.0,15.0,6.0,13.0,15.0
145,Wright State,19.0,13.0,23.0,21.0,11.0,22.0,20.0,25.0,21.0,...,13.0,15.0,20.0,13.0,12.0,10.0,14.0,6.0,14.0,15.0


In [18]:
df_records_unmatched = df_records.loc[~df_records['team_id'].isin(team_guess['team_id']), 
                                      :].reset_index(drop=True)

df_records_unmatched

,team_id,team_name,2011_wins,2012_wins,2013_wins,2014_wins,2015_wins,2016_wins,2017_wins,2018_wins,...,2013_losses,2014_losses,2015_losses,2016_losses,2017_losses,2018_losses,2019_losses,2021_losses,2022_losses,2023_losses
0,1101,Abilene Chr,NaN,NaN,NaN,2.0,7.0,9.0,9.0,12.0,...,NaN,19.0,21.0,18.0,16.0,15.0,6.0,4.0,10.0,17.0
1,1106,Alabama St,15.0,10.0,9.0,16.0,17.0,12.0,7.0,8.0,...,22.0,12.0,10.0,17.0,23.0,23.0,19.0,14.0,21.0,23.0
2,1107,SUNY Albany,16.0,19.0,24.0,18.0,24.0,23.0,20.0,21.0,...,10.0,14.0,8.0,8.0,13.0,10.0,20.0,9.0,18.0,23.0
3,1108,Alcorn St,4.0,7.0,9.0,9.0,5.0,13.0,15.0,7.0,...,23.0,19.0,25.0,15.0,14.0,21.0,21.0,13.0,16.0,13.0
4,1110,American Univ,22.0,20.0,10.0,20.0,17.0,12.0,8.0,6.0,...,20.0,12.0,16.0,19.0,22.0,24.0,15.0,6.0,22.0,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,1464,Youngstown St,7.0,14.0,14.0,12.0,8.0,9.0,11.0,6.0,...,15.0,17.0,21.0,21.0,21.0,24.0,20.0,12.0,14.0,9.0
143,1465,Cal Baptist,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,10.0,15.0,16.0
144,1470,Tarleton St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,17.0,16.0
145,1472,St Thomas MN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.0,14.0


In [19]:
from fuzzywuzzy.fuzz import token_sort_ratio
from fuzzywuzzy import process

def find_closest_team_match(team_id, team_name, df_sr_unmatched=df_sr_unmatched):
    df_sr_unmatched = df_sr_unmatched.copy()
    
    # sr_unmatched = df_sr_unmatched.loc[df_sr_unmatched['seasons'] == seasons, 'School']
    
    sr_unmatched = df_sr_unmatched.loc[:, 'School']
    
    # if sr_unmatched.shape[0] == 0:
    #     return [team_id, team_name, seasons, None, 0]
    
    return [team_id, team_name, *process.extract(team_name, 
                                                 sr_unmatched, 
                                                 scorer=token_sort_ratio, 
                                                 limit=1)[0][:2]]

find_closest_team_match(1101, 'Abilene Chr')

[1101, 'Abilene Chr', 'Abilene Christian', 79]

In [20]:
team_guess2 = pd.DataFrame(
    [find_closest_team_match(team_id, team_name) 
     for team_id, team_name, in np.array(df_records_unmatched[['team_id', 'team_name']])], 
    columns = ['team_id', 'kaggle_team', 'sr_team', 'score']
).sort_values(['score'], ignore_index=True)

team_guess2

,team_id,kaggle_team,sr_team,score
0,1236,PFW,Purdue-Fort Wayne,20
1,1416,UCF,UTSA,29
2,1419,ULM,UTSA,29
3,1433,VCU,UTSA,29
4,1443,WKU,Milwaukee,33
...,...,...,...,...
142,1253,Long Beach St,Long Beach State,90
143,1308,New Mexico St,New Mexico State,90
144,1240,Jacksonville St,Jacksonville State,91
145,1477,TX A&M Commerce,Texas A&M-Commerce,91


In [21]:
team_guess = pd.concat(
    [team_guess, team_guess2], ignore_index=True
).sort_values(['score'], ignore_index=True)

team_guess

,team_id,kaggle_team,sr_team,score
0,1236,PFW,Purdue-Fort Wayne,20
1,1419,ULM,UTSA,29
2,1416,UCF,UTSA,29
3,1433,VCU,UTSA,29
4,1420,UMBC,American,33
...,...,...,...,...
360,1317,North Texas,North Texas,100
361,1316,North Florida,North Florida,100
362,1315,North Dakota,North Dakota,100
363,1312,NJIT,NJIT,100


In [22]:
team_guess.loc[team_guess['kaggle_team'] == 'SMU', ['sr_team', 'score']] = 'Southern Methodist', 100
team_guess.loc[team_guess['kaggle_team'] == 'UCF', ['sr_team', 'score']] = 'Central Florida', 100
team_guess.loc[team_guess['kaggle_team'] == 'VCU', ['sr_team', 'score']] = 'Virginia Commonwealth', 100
team_guess.loc[team_guess['kaggle_team'] == 'LSU', ['sr_team', 'score']] = 'Louisiana State', 100
team_guess.loc[team_guess['kaggle_team'] == 'USC', ['sr_team', 'score']] = 'Southern California', 100
team_guess.loc[team_guess['kaggle_team'] == 'WKU', ['sr_team', 'score']] = 'Western Kentucky', 100
team_guess.loc[team_guess['kaggle_team'] == 'MTSU', ['sr_team', 'score']] = 'Middle Tennessee', 100
team_guess.loc[team_guess['kaggle_team'] == 'SIUE', ['sr_team', 'score']] = 'SIU Edwardsville', 100
team_guess.loc[team_guess['kaggle_team'] == 'PFW', ['sr_team', 'score']] = 'Purdue-Fort Wayne', 100
team_guess.loc[team_guess['kaggle_team'] == 'UNLV', ['sr_team', 'score']] = 'Nevada-Las Vegas', 100
team_guess.loc[team_guess['kaggle_team'] == 'ULM', ['sr_team', 'score']] = 'Louisiana-Monroe', 100
team_guess.loc[team_guess['kaggle_team'] == 'LIU Brooklyn', ['sr_team', 'score']] = 'Long Island University', 100
team_guess.loc[team_guess['kaggle_team'] == 'Missouri KC', ['sr_team', 'score']] = 'Kansas City', 100
team_guess.loc[team_guess['kaggle_team'] == 'NC A&T', ['sr_team', 'score']] = 'North Carolina A&T', 100
team_guess.loc[team_guess['kaggle_team'] == 'UTRGV', ['sr_team', 'score']] = 'Texas-Rio Grande Valley', 100
team_guess.loc[team_guess['kaggle_team'] == 'UMBC', ['sr_team', 'score']] = 'Maryland-Baltimore County', 100
team_guess.loc[team_guess['kaggle_team'] == 'ETSU', ['sr_team', 'score']] = 'East Tennessee State', 100
team_guess.loc[team_guess['kaggle_team'] == 'Penn', ['sr_team', 'score']] = 'Pennsylvania', 100
team_guess.loc[team_guess['kaggle_team'] == 'UT San Antonio', ['sr_team', 'score']] = 'UTSA', 100
team_guess.loc[team_guess['kaggle_team'] == 'SC Upstate', ['sr_team', 'score']] = 'South Carolina Upstate', 100
team_guess.loc[team_guess['kaggle_team'] == 'NC Central', ['sr_team', 'score']] = 'North Carolina Central', 100
team_guess.loc[team_guess['kaggle_team'] == 'E Michigan', ['sr_team', 'score']] = 'Eastern Michigan', 100
team_guess.loc[team_guess['kaggle_team'] == 'Southern Univ', ['sr_team', 'score']] = 'Southern', 100
team_guess.loc[team_guess['kaggle_team'] == 'Florida Intl', ['sr_team', 'score']] = 'Florida International', 100
team_guess.loc[team_guess['kaggle_team'] == 'S Dakota St', ['sr_team', 'score']] = 'South Dakota State', 100
team_guess.loc[team_guess['kaggle_team'] == 'N Dakota St', ['sr_team', 'score']] = 'North Dakota State', 100
team_guess.loc[team_guess['kaggle_team'] == 'N Kentucky', ['sr_team', 'score']] = 'Northern Kentucky', 100
team_guess.loc[team_guess['kaggle_team'] == 'W Illinois', ['sr_team', 'score']] = 'Western Illinois', 100
team_guess.loc[team_guess['kaggle_team'] == 'TAM C. Christi', ['sr_team', 'score']] = 'Texas A&M-Corpus Christi', 100
team_guess.loc[team_guess['kaggle_team'] == 'Central Conn', ['sr_team', 'score']] = 'Central Connecticut State', 100
team_guess.loc[team_guess['kaggle_team'] == 'N Colorado', ['sr_team', 'score']] = 'Northern Colorado', 100
team_guess.loc[team_guess['kaggle_team'] == 'N Illinois', ['sr_team', 'score']] = 'Northern Illinois', 100
team_guess.loc[team_guess['kaggle_team'] == 'S Illinois', ['sr_team', 'score']] = 'Southern Illinois', 100
team_guess.loc[team_guess['kaggle_team'] == 'SE Missouri St', ['sr_team', 'score']] = 'Southeast Missouri State', 100
team_guess.loc[team_guess['kaggle_team'] == 'Ga Southern', ['sr_team', 'score']] = 'Georgia Southern', 100
team_guess.loc[team_guess['kaggle_team'] == 'E Washington', ['sr_team', 'score']] = 'Eastern Washington', 100
team_guess.loc[team_guess['kaggle_team'] == 'SE Louisiana', ['sr_team', 'score']] = 'Southeastern Louisiana', 100

team_guess.sort_values(['score'], inplace=True, ignore_index=True)

team_guess

,team_id,kaggle_team,sr_team,score
0,1140,BYU,Brigham Young,38
1,1245,Kent,Kent State,57
2,1271,MD E Shore,Maryland-Eastern Shore,62
3,1262,MA Lowell,Massachusetts-Lowell,62
4,1372,SF Austin,Stephen F Austin,64
...,...,...,...,...
360,1196,Florida,Florida,100
361,1193,Fairfield,Fairfield,100
362,1210,Georgia Tech,Georgia Tech,100
363,1127,Binghamton,Binghamton,100


In [ ]:
# team_guess.loc[team_guess['kaggle_team'] == '', ['sr_team', 'score']] = '', 100
# team_guess.loc[team_guess['kaggle_team'] == '', ['sr_team', 'score']] = '', 100

In [23]:
team_guess.loc[70:79, ['kaggle_team', 'sr_team', 'score']]

,kaggle_team,sr_team,score
70,Wichita St,Wichita State,87
71,Chicago St,Chicago State,87
72,Florida St,Florida State,87
73,Arizona St,Arizona State,87
74,Alabama St,Alabama State,87
75,Indiana St,Indiana State,87
76,Jackson St,Jackson State,87
77,Norfolk St,Norfolk State,87
78,Montana St,Montana State,87
79,Colorado St,Colorado State,88


In [24]:
df_sr_teams.loc[df_sr_teams['School'].str.contains('ern louis', case=False), :]

,School,2011_W,2012_W,2013_W,2014_W,2015_W,2016_W,2017_W,2018_W,2019_W,...,2013_L,2014_L,2015_L,2016_L,2017_L,2018_L,2019_L,2021_L,2022_L,2023_L
281,Southeastern Louisiana,15.0,12.0,13.0,12.0,9.0,12.0,16.0,22.0,17.0,...,18.0,18.0,23.0,21.0,16.0,12.0,16.0,18.0,15.0,14.0


In [25]:
team_guess.loc[team_guess['sr_team'].isin(team_guess.loc[team_guess['sr_team'].duplicated(), 'sr_team']), :]

,team_id,kaggle_team,sr_team,score


In [26]:
team_guess.sort_values(['score'], inplace=True, ignore_index=True)

team_guess.head(25)

,team_id,kaggle_team,sr_team,score
0,1140,BYU,Brigham Young,38
1,1245,Kent,Kent State,57
2,1271,MD E Shore,Maryland-Eastern Shore,62
3,1262,MA Lowell,Massachusetts-Lowell,62
4,1372,SF Austin,Stephen F Austin,64
5,1290,MS Valley St,Mississippi Valley State,67
6,1170,CS Sacramento,Sacramento State,69
7,1178,Detroit,Detroit Mercy,70
8,1260,Loyola-Chicago,Loyola IL,70
9,1168,CS Fullerton,Cal State Fullerton,71


In [27]:
team_guess.insert(
    team_guess.columns.get_loc('sr_team') + 1, 'url_team', 
    team_guess['sr_team'].str.lower().str.replace(
        ' & ', '-', regex=False).str.replace(
        '&', '', regex=False).str.replace(
        ' ', '-', regex=False)
)

team_guess

,team_id,kaggle_team,sr_team,url_team,score
0,1140,BYU,Brigham Young,brigham-young,38
1,1245,Kent,Kent State,kent-state,57
2,1271,MD E Shore,Maryland-Eastern Shore,maryland-eastern-shore,62
3,1262,MA Lowell,Massachusetts-Lowell,massachusetts-lowell,62
4,1372,SF Austin,Stephen F Austin,stephen-f-austin,64
...,...,...,...,...,...
360,1316,North Florida,North Florida,north-florida,100
361,1315,North Dakota,North Dakota,north-dakota,100
362,1332,Oregon,Oregon,oregon,100
363,1126,Bethune-Cookman,Bethune-Cookman,bethune-cookman,100


In [28]:
team_guess.loc[team_guess['sr_team'].str.contains('&', regex=False), :]

,team_id,kaggle_team,sr_team,url_team,score
107,1477,TX A&M Commerce,Texas A&M-Commerce,texas-am-commerce,91
109,1299,NC A&T,North Carolina A&T,north-carolina-at,100
126,1394,TAM C. Christi,Texas A&M-Corpus Christi,texas-am-corpus-christi,100
163,1456,William & Mary,William & Mary,william-mary,100
197,1197,Florida A&M,Florida A&M,florida-am,100
256,1105,Alabama A&M,Alabama A&M,alabama-am,100
304,1401,Texas A&M,Texas A&M,texas-am,100


In [29]:
team_guess.sample(15)

,team_id,kaggle_team,sr_team,url_team,score
163,1456,William & Mary,William & Mary,william-mary,100
115,1420,UMBC,Maryland-Baltimore County,maryland-baltimore-county,100
65,1293,Murray St,Murray State,murray-state,86
339,1309,New Orleans,New Orleans,new-orleans,100
59,1453,WI Green Bay,Green Bay,green-bay,86
322,1304,Nebraska,Nebraska,nebraska,100
68,1286,Montana St,Montana State,montana-state,87
313,1390,Stanford,Stanford,stanford,100
76,1270,McNeese St,McNeese State,mcneese-state,87
310,1393,Syracuse,Syracuse,syracuse,100


In [30]:
team_guess.loc[team_guess['kaggle_team'] == 'NE Omaha', 'url_team'] = 'nebraska-omaha'
team_guess.loc[team_guess['kaggle_team'] == 'Citadel', 'url_team'] = 'citadel'
team_guess.loc[team_guess['kaggle_team'] == 'Houston Chr', 'url_team'] = 'houston-baptist'
team_guess.loc[team_guess['kaggle_team'] == 'Ark Little Rock', 'url_team'] = 'arkansas-little-rock'
team_guess.loc[team_guess['kaggle_team'] == 'St Thomas MN', 'url_team'] = 'st-thomas-mn'
team_guess.loc[team_guess['kaggle_team'] == 'SIUE', 'url_team'] = 'southern-illinois-edwardsville'
team_guess.loc[team_guess['kaggle_team'] == 'UTRGV', 'url_team'] = 'texas-pan-american'
team_guess.loc[team_guess['kaggle_team'] == 'UT San Antonio', 'url_team'] = 'texas-san-antonio'
team_guess.loc[team_guess['kaggle_team'] == 'Missouri KC', 'url_team'] = 'missouri-kansas-city'
team_guess.loc[team_guess['kaggle_team'] == 'VMI', 'url_team'] = 'virginia-military-institute'
team_guess.loc[team_guess['kaggle_team'] == 'UTEP', 'url_team'] = 'texas-el-paso'
team_guess.loc[team_guess['kaggle_team'] == 'Utah Tech', 'url_team'] = 'dixie-state'
team_guess.loc[team_guess['kaggle_team'] == 'UT Arlington', 'url_team'] = 'texas-arlington'
team_guess.loc[team_guess['kaggle_team'] == 'UNC Wilmington', 'url_team'] = 'north-carolina-wilmington'
team_guess.loc[team_guess['kaggle_team'] == 'UNC Greensboro', 'url_team'] = 'north-carolina-greensboro'
team_guess.loc[team_guess['kaggle_team'] == 'UNC Asheville', 'url_team'] = 'north-carolina-asheville'
team_guess.loc[team_guess['kaggle_team'] == 'Louisiana', 'url_team'] = 'louisiana-lafayette'
team_guess.loc[team_guess['kaggle_team'] == 'UC San Diego', 'url_team'] = 'california-san-diego'
team_guess.loc[team_guess['kaggle_team'] == 'UC Irvine', 'url_team'] = 'california-irvine'
team_guess.loc[team_guess['kaggle_team'] == 'UAB', 'url_team'] = 'alabama-birmingham'
team_guess.loc[team_guess['kaggle_team'] == 'UC Davis', 'url_team'] = 'california-davis'
team_guess.loc[team_guess['kaggle_team'] == 'UC Santa Barbara', 'url_team'] = 'california-santa-barbara'
team_guess.loc[team_guess['kaggle_team'] == 'PFW', 'url_team'] = 'ipfw'
team_guess.loc[team_guess['kaggle_team'] == 'TCU', 'url_team'] = 'texas-christian'
team_guess.loc[team_guess['kaggle_team'] == 'NC State', 'url_team'] = 'north-carolina-state'
team_guess.loc[team_guess['kaggle_team'] == 'UC Riverside', 'url_team'] = 'california-riverside'

team_guess

,team_id,kaggle_team,sr_team,url_team,score
0,1140,BYU,Brigham Young,brigham-young,38
1,1245,Kent,Kent State,kent-state,57
2,1271,MD E Shore,Maryland-Eastern Shore,maryland-eastern-shore,62
3,1262,MA Lowell,Massachusetts-Lowell,massachusetts-lowell,62
4,1372,SF Austin,Stephen F Austin,stephen-f-austin,64
...,...,...,...,...,...
360,1316,North Florida,North Florida,north-florida,100
361,1315,North Dakota,North Dakota,north-dakota,100
362,1332,Oregon,Oregon,oregon,100
363,1126,Bethune-Cookman,Bethune-Cookman,bethune-cookman,100


In [31]:
team_guess.to_pickle('../resources/data/preprocessed/kaggle_to_sr.pkl')

'Done'

'Done'

In [32]:
df_start = pd.read_csv('../resources/data/unprocessed/MSeasons.csv')

df_start

,Season,DayZero,RegionW,RegionX,RegionY,RegionZ
0,1985,10/29/1984,East,West,Midwest,Southeast
1,1986,10/28/1985,East,Midwest,Southeast,West
2,1987,10/27/1986,East,Southeast,Midwest,West
3,1988,11/2/1987,East,Midwest,Southeast,West
4,1989,10/31/1988,East,West,Midwest,Southeast
5,1990,10/30/1989,East,Midwest,Southeast,West
6,1991,10/29/1990,East,Southeast,Midwest,West
7,1992,11/4/1991,East,West,Midwest,Southeast
8,1993,11/2/1992,East,Midwest,Southeast,West
9,1994,11/1/1993,East,Southeast,Midwest,West


In [33]:
import datetime as dt

In [34]:
team_guess['kaggle_team'][:10]

0               BYU
1              Kent
2        MD E Shore
3         MA Lowell
4         SF Austin
5      MS Valley St
6     CS Sacramento
7           Detroit
8    Loyola-Chicago
9      CS Fullerton
Name: kaggle_team, dtype: object

In [35]:
import time

time.sleep(0)

'Done'

'Done'

In [36]:
temp = team_guess.loc[team_guess['team_id'] == 1386, :]

temp

,team_id,kaggle_team,sr_team,url_team,score
15,1386,St Joseph's PA,Saint Josephs,saint-josephs,74


In [37]:
%%time

schedules = []
error_tracker = []
for index, (team_id, team, url_team) in enumerate(np.array(team_guess[['team_id', 'kaggle_team', 'url_team']])):    
    for season in kaggle_teams_seasons_mapping[team]:
        print(f'{index}: {team_id} {team} {season}')
        
        try:
            schedule = pd.read_html(
                f'https://www.sports-reference.com/cbb/schools/{url_team}/men/{season}-starters.html'
            )[0]
        except Exception as e:
            print(f'WARNING: {team_id} {team} {season} ran into an error: {e}')
            error_tracker.append((team_id, team, season, e))
            time.sleep(3.1)
            continue

        schedule = schedule.iloc[:, [1, 5, 12]]

        schedule.columns = ['date', 'opponent', 'starters']

        schedule.insert(0, 'season', season)

        schedule['date'] = schedule['date'].str[4:]

        schedule.insert(
            2, 'day_number', (
                pd.to_datetime(schedule['date']) - 
                pd.to_datetime(df_start.loc[df_start['Season'] == season, 'DayZero'].iloc[0])
            ).dt.days
        )

        schedule.insert(
            3, 'backup_day_number', schedule['day_number'] - 1
        )

        schedule.drop(columns=['date'], inplace=True)

        schedule.insert(
            4, 'team_id', team_id
        )

        schedule.insert(
            5, 'team', team
        )
        
        schedule = schedule.loc[~schedule['starters'].isna(), :].reset_index(drop=True)

        schedule[['starter1', 'starter2', 'starter3', 'starter4', 'starter5']] = schedule['starters'].str.replace(
            '\xa0', '', regex=False).str.replace(
            ' ', '', regex=False).str.replace(
            '.', '. ', regex=False).str.split(
            '·').tolist()

        schedule.drop(columns=['starters'], inplace=True)

        schedules.append(schedule)

        time.sleep(3.1)

df_schedules = pd.concat(schedules, ignore_index=True)

df_schedules

0: 1140 BYU 2011
0: 1140 BYU 2012
0: 1140 BYU 2013
0: 1140 BYU 2014
0: 1140 BYU 2015
0: 1140 BYU 2016
0: 1140 BYU 2017
0: 1140 BYU 2018
0: 1140 BYU 2019
0: 1140 BYU 2021
0: 1140 BYU 2022
0: 1140 BYU 2023
1: 1245 Kent 2011
1: 1245 Kent 2012
1: 1245 Kent 2013
1: 1245 Kent 2014
1: 1245 Kent 2015
1: 1245 Kent 2016
1: 1245 Kent 2017
1: 1245 Kent 2018
1: 1245 Kent 2019
1: 1245 Kent 2021
1: 1245 Kent 2022
1: 1245 Kent 2023
2: 1271 MD E Shore 2011
2: 1271 MD E Shore 2012
2: 1271 MD E Shore 2013
2: 1271 MD E Shore 2014
2: 1271 MD E Shore 2015
2: 1271 MD E Shore 2016
2: 1271 MD E Shore 2017
2: 1271 MD E Shore 2018
2: 1271 MD E Shore 2019
2: 1271 MD E Shore 2022
2: 1271 MD E Shore 2023
3: 1262 MA Lowell 2014
3: 1262 MA Lowell 2015
3: 1262 MA Lowell 2016
3: 1262 MA Lowell 2017
3: 1262 MA Lowell 2018
3: 1262 MA Lowell 2019
3: 1262 MA Lowell 2021
3: 1262 MA Lowell 2022
3: 1262 MA Lowell 2023
4: 1372 SF Austin 2011
4: 1372 SF Austin 2012
4: 1372 SF Austin 2013
4: 1372 SF Austin 2014
4: 1372 SF Austin

,season,day_number,backup_day_number,opponent,team_id,team,starter1,starter2,starter3,starter4,starter5
0,2011,11,10,Fresno State,1140,BYU,C. Abouo,C. Collinsworth,B. Davies,J. Emery,J. Fredette
1,2011,16,15,Utah State,1140,BYU,C. Abouo,C. Collinsworth,J. Emery,J. Fredette,N. Hartsock
2,2011,19,18,Chicago State,1140,BYU,C. Abouo,C. Collinsworth,J. Emery,J. Fredette,N. Hartsock
3,2011,22,21,Mississippi Valley State,1140,BYU,C. Abouo,C. Collinsworth,J. Emery,J. Fredette,N. Hartsock
4,2011,25,24,South Florida,1140,BYU,K. Collinsworth,B. Davies,J. Emery,J. Fredette,N. Hartsock
...,...,...,...,...,...,...,...,...,...,...,...
134094,2023,107,106,Winthrop,1205,Gardner Webb,L. Dufeal,D. Nicholas,K. Reid,A. Selden,J. Soumaoro
134095,2023,110,109,UNC Asheville,1205,Gardner Webb,L. Dufeal,D. Nicholas,K. Reid,A. Selden,J. Soumaoro
134096,2023,115,114,Longwood,1205,Gardner Webb,L. Dufeal,D. Nicholas,K. Reid,J. Roghelia,A. Selden
134097,2023,117,116,South Carolina Upstate,1205,Gardner Webb,L. Dufeal,D. Nicholas,K. Reid,A. Selden,L. Stieber


In [38]:
df_errors = pd.DataFrame(error_tracker, columns=['team_id', 'team_name', 'season', 'error'])

df_errors['error'] = df_errors['error'].astype(str)

df_errors

,team_id,team_name,season,error
0,1260,Loyola-Chicago,2014,HTTP Error 520:
1,1260,Loyola-Chicago,2022,HTTP Error 520:
2,1303,NE Omaha,2012,HTTP Error 404: Not Found


In [39]:
df_errors['url_team'] = df_errors.merge(team_guess, how='left', on=['team_id'])['url_team']

df_errors

,team_id,team_name,season,error,url_team
0,1260,Loyola-Chicago,2014,HTTP Error 520:,loyola-il
1,1260,Loyola-Chicago,2022,HTTP Error 520:,loyola-il
2,1303,NE Omaha,2012,HTTP Error 404: Not Found,nebraska-omaha


In [40]:
%%time

schedules = []
error_tracker = []
for index, (team_id, team, url_team, season) in enumerate(np.array(df_errors[['team_id', 'team_name', 'url_team', 'season']])):    
    print(f'{index}: {team_id} {team} {season}')

    try:
        schedule = pd.read_html(
            f'https://www.sports-reference.com/cbb/schools/{url_team}/men/{season}-starters.html'
        )[0]
    except Exception as e:
        print(f'WARNING: {team_id} {team} {season} ran into an error: {e}')
        error_tracker.append((team_id, team, season, e))
        time.sleep(3.1)
        continue

    schedule = schedule.iloc[:, [1, 5, 12]]

    schedule.columns = ['date', 'opponent', 'starters']

    schedule.insert(0, 'season', season)

    schedule['date'] = schedule['date'].str[4:]

    schedule.insert(
        2, 'day_number', (
            pd.to_datetime(schedule['date']) - 
            pd.to_datetime(df_start.loc[df_start['Season'] == season, 'DayZero'].iloc[0])
        ).dt.days
    )

    schedule.insert(
        3, 'backup_day_number', schedule['day_number'] - 1
    )

    schedule.drop(columns=['date'], inplace=True)

    schedule.insert(
        4, 'team_id', team_id
    )

    schedule.insert(
        5, 'team', team
    )

    schedule = schedule.loc[~schedule['starters'].isna(), :].reset_index(drop=True)

    schedule[['starter1', 'starter2', 'starter3', 'starter4', 'starter5']] = schedule['starters'].str.replace(
        '\xa0', '', regex=False).str.replace(
        ' ', '', regex=False).str.replace(
        '.', '. ', regex=False).str.split(
        '·').tolist()

    schedule.drop(columns=['starters'], inplace=True)

    schedules.append(schedule)

    time.sleep(3.1)

df_schedules2 = pd.concat(schedules, ignore_index=True)

df_schedules2

0: 1260 Loyola-Chicago 2014
1: 1260 Loyola-Chicago 2022
2: 1303 NE Omaha 2012
CPU times: total: 172 ms
Wall time: 9.84 s


,season,day_number,backup_day_number,opponent,team_id,team,starter1,starter2,starter3,starter4,starter5
0,2014,4,3,Milwaukee,1260,Loyola-Chicago,J. Crisman,M. Doyle,M. O'Leary,C. Thomas,J. White
1,2014,8,7,Tennessee Tech,1260,Loyola-Chicago,J. Crisman,M. Doyle,M. O'Leary,C. Thomas,J. White
2,2014,12,11,Tulane,1260,Loyola-Chicago,J. Crisman,M. Doyle,N. Osborne,C. Thomas,J. White
3,2014,15,14,Rockhurst,1260,Loyola-Chicago,J. Crisman,M. Doyle,N. Osborne,C. Thomas,J. White
4,2014,18,17,Portland State,1260,Loyola-Chicago,J. Crisman,M. Doyle,N. Osborne,C. Thomas,J. White
...,...,...,...,...,...,...,...,...,...,...,...
60,2022,117,116,Northern Iowa,1260,Loyola-Chicago,T. Hall,C. Knight,B. Norris,A. Uguak,L. Williamson
61,2022,123,122,Bradley,1260,Loyola-Chicago,T. Hall,C. Knight,B. Norris,A. Uguak,L. Williamson
62,2022,124,123,Northern Iowa,1260,Loyola-Chicago,T. Hall,C. Knight,B. Norris,A. Uguak,L. Williamson
63,2022,125,124,Drake,1260,Loyola-Chicago,T. Hall,C. Knight,B. Norris,A. Uguak,L. Williamson


In [41]:
error_tracker

[(1303, 'NE Omaha', 2012, <HTTPError 404: 'Not Found'>)]

In [42]:
df_schedules = pd.concat(
    [df_schedules, df_schedules2], 
    ignore_index=True
).sort_values(['season', 'day_number', 'team_id'], ignore_index=True)

df_schedules

,season,day_number,backup_day_number,opponent,team_id,team,starter1,starter2,starter3,starter4,starter5
0,2011,7,6,UC Irvine,1228,Illinois,B. Cole,M. Davis,D. McCamey,D. Richardson,M. Tisdale
1,2011,7,6,Seattle,1268,Maryland,A. Bowie,D. Gregory,S. Mosley,C. Tucker,J. Williams
2,2011,7,6,Texas,1298,Navy,J. Avila,O. Avworo,R. Garcia,J. Sugars,M. Veazey
3,2011,7,6,Rhode Island,1338,Pittsburgh,G. Brown,A. Gibbs,G. McGhee,B. Wanamaker,T. Zanna
4,2011,7,6,Pittsburgh,1348,Rhode Island,R. Brooks,D. James,M. Jones,W. Martell,A. Richmond
...,...,...,...,...,...,...,...,...,...,...,...
134159,2023,148,147,Utah Valley,1412,UAB,T. Brewer,K. Buffen,T. Jemison,T. Lovan,J. Walker
134160,2023,148,147,UAB,1430,Utah Valley,A. Bandaogo,L. Darthard,T. Fuller,J. Harmon,T. Woodbury
134161,2023,148,147,North Texas,1458,Wisconsin,S. Crowl,C. Essegian,C. Hepburn,M. Klesmit,T. Wahl
134162,2023,150,149,UAB,1317,North Texas,K. Huntsberry,R. Jones,T. Perry,A. Scott,M. Sissoko


In [43]:
# adjust opponent id based on changes to Sports Reference team names made earlier

df_schedules['opponent'] = df_schedules['opponent'].str.replace('(', '', regex=False).str.replace(
    ')', '', regex=False).str.replace(
    "'", '', regex=False).str.replace(
    '.', '', regex=False)

df_schedules['opponent_id'] = df_schedules.merge(
    team_guess, how='left', left_on=['opponent'], right_on=['sr_team']
)['team_id_y']

In [47]:
df_schedules.loc[df_schedules['opponent_id'].isna(), 'opponent'].unique()

array(['Millikin', 'Vassar', 'Texas A&M-Kingsville', 'Concordia IL',
       'Piedmont', 'Erskine', 'Florida Memorial', 'North Greenville',
       'Southern Illinois-Edwardsville', 'Georgia Southwestern', 'Lyon',
       'Virginia Intermont', 'West Virginia Tech', 'Oglethorpe',
       'Truman State', 'Asbury', 'Tennessee Temple', 'Tabor College',
       'Johnson & Wales NC', 'Waldorf', 'Henderson State', 'St Thomas TX',
       'Occidental', 'Brewton-Parker', 'Millsaps', 'Howard Payne',
       'Spring Hill', 'College of Idaho', 'Sul Ross State', 'MacMurray',
       'Huntington', 'West Florida', 'Texas Lutheran', 'Huston-Tillotson',
       'Indiana-Northwest', 'Randolph', 'St Francis IL', 'Talladega',
       'Edward Waters', 'Western Colorado', 'Marian IN', 'Hendrix',
       'LaGrange', 'Washington Adventist', 'Bluefield State',
       'Oakland City', 'Mary Washington', 'Ferrum', 'Farmingdale State',
       'Eastern Oregon', 'West Alabama', 'Austin College', 'Lesley',
       'Emory & Henry

In [48]:
df_schedules.to_pickle('../resources/data/preprocessed/schedules.pkl')

'Done'

'Done'